### Michael Felzan

### GIS 5572 -- Final Project

### Real-time Radio Station Audio Sampling, Processing, and Logging as a Proxy for Spatiotemporal Popular Music Trends

### 𓀂 𓀂 𓀂 𓀂 𓀂 𓀂 𓀂 𓀂 𓀂 𓀂 𓀂 𓀂 𓀂 𓀂 𓀂 𓀂 𓀂 𓀂 𓀂 𓀂 𓀂 𓀂 𓀂 𓀂 𓀂 𓀂 𓀂 𓀂 𓀂 𓀂 𓀂 𓀂 𓀂 𓀂 𓀂 𓀂 𓀂 𓀂 𓀂 𓀂 𓀂 𓀂 𓀂 𓀂 𓀂 𓀂 𓀂 𓀂 𓀂 𓀂 𓀂 𓀂 𓀂 𓀂 𓀂 𓀂 𓀂 𓀂 𓀂 𓀂 𓀂 𓀂 𓀂 𓀂 𓀂 𓀂 𓀂 

## Importing Packages

##### NOTE: in order to import 'acrcloud' properly, user must go to terminal, navigate to base of project folder repo, and and type 'sudo python setup.py install'

In [131]:
import time
import datetime
import vlc
import requests
import urllib.parse
from urllib.parse import parse_qs
import os
import csv
import shutil
from shutil import copyfile
import sys
import json
import glob
from acrcloud.recognizer import ACRCloudRecognizer
from acrcloud.recognizer import ACRCloudRecognizeType
from itertools import islice
from bs4 import BeautifulSoup
import subprocess
import pandas as pd

# The following packages were not used in this code, but
# if the program was to be expanded, using OAuth1 / 
# Discog's more advanced API might be required... 

    #from requests_oauthlib import OAuth1
    #from requests_oauthlib import OAuth1Session
    #from flask import Flask

## User Inputs -- Working Directory Path / API Key Inputs

##### Path to base of user's working folder/ repo:

In [132]:
workindir = r'/Users/michaelfelzan/Desktop/RadioTestRun'

##### User must input their personal ACR Cloud Access and Secret keys here:

In [133]:
acrcloud_accesskey = '4554490a1c83c19ea6745ed6bfbbea7d'
acrcloud_secretkey = 'HcoVKztgUltoNAwlPxpWoIJzt86HoKk9KTiByPTl'

##### User must input their personal Discogs Access and Secret keys here:

In [134]:
client_key = 'gQylLmyvGPfyGEvmFyRx'
client_secret = 'KJnItkPttweOtFSkqwAiTTCWJsOnIobH'

# ∿ ∿ ∿ ∿ ∿ ∿ ∿ ∿ ∿ ∿ ∿ ∿ ∿ ∿ ∿ ∿ ∿ ∿ ∿ ∿ ∿ ∿ ∿ ∿ ∿ ∿ ∿ ∿ ∿ ∿ ∿ ∿ ∿ ∿ ∿ ∿ ∿ ∿ ∿ ∿ ∿ ∿ 

# ∿ ∿ ∿ ∿ ∿ ∿ ∿ ∿ ∿ ∿ ∿ ∿ ∿ ∿ ∿ ∿ ∿ ∿ ∿ ∿ ∿ ∿ ∿ ∿ ∿ ∿ ∿ ∿ ∿ ∿ ∿ ∿ ∿ ∿ ∿ ∿ ∿ ∿ ∿ ∿ ∿ ∿ 

# { { { { { { { {   {  { { { { { {  { { {  { { { { { {_  Section A.   _} } } } } } } } } }  } } }  } } } } } } } } } } } }   }   }    } 

## Setting up radio station dictonary structure,

#### with each radio station's folder name (within the 'STATIONS' folder) as a key, and the value being a nested dictonary, containing: the dir path to that folder, the path to that folder's '.txt' file, and the online radio stream URL (contained within the .txt file)

##### creating list of station (folder) names:

In [135]:
os.chdir(os.path.join(workindir,"FiveStations"))

stationfolders_wdsstore = os.listdir()

# making sure the '.DS_Store' file isnt included
stationfolders = []
for item in stationfolders_wdsstore:
    if item=='.DS_Store':
        pass
    else:
        stationfolders.append(item)
sortedstations = sorted(stationfolders)
sortedstations[:8]

['MN_KCMP', 'MN_KQRS', 'MN_KSTP', 'MN_KUOM', 'MN_KXXR']

##### creating list of the pathes to each station folder:

In [136]:
pathes_to_folders = []
pathes_to_txts = []

for foldername in sortedstations:
    iter_path_to_folder = os.path.join(workindir,'FiveStations',foldername)
    iter_path_to_txts = os.path.join(iter_path_to_folder,(foldername+'.txt'))
    pathes_to_folders.append(iter_path_to_folder)
    pathes_to_txts.append(iter_path_to_txts)
pathes_to_folders[:5]

['/Users/michaelfelzan/Desktop/RadioTestRun/FiveStations/MN_KCMP',
 '/Users/michaelfelzan/Desktop/RadioTestRun/FiveStations/MN_KQRS',
 '/Users/michaelfelzan/Desktop/RadioTestRun/FiveStations/MN_KSTP',
 '/Users/michaelfelzan/Desktop/RadioTestRun/FiveStations/MN_KUOM',
 '/Users/michaelfelzan/Desktop/RadioTestRun/FiveStations/MN_KXXR']

##### creating list of the pathes to each station's respective .txt file:

In [137]:
pathes_to_txts[:5]

['/Users/michaelfelzan/Desktop/RadioTestRun/FiveStations/MN_KCMP/MN_KCMP.txt',
 '/Users/michaelfelzan/Desktop/RadioTestRun/FiveStations/MN_KQRS/MN_KQRS.txt',
 '/Users/michaelfelzan/Desktop/RadioTestRun/FiveStations/MN_KSTP/MN_KSTP.txt',
 '/Users/michaelfelzan/Desktop/RadioTestRun/FiveStations/MN_KUOM/MN_KUOM.txt',
 '/Users/michaelfelzan/Desktop/RadioTestRun/FiveStations/MN_KXXR/MN_KXXR.txt']

##### creating dictionary structure, where each station (folder) name is the key, and the value is a nested dictionary, which contains key/value pairs for each station's folder pathname, path to .txt file, and radio station URL stream (accessed from .txt file):

In [138]:
station_dict = {}

for foldername in sortedstations:
    iter_path_to_folder = os.path.join(workindir,
                                       'FiveStations',
                                       foldername)
    iter_path_to_txts = os.path.join(iter_path_to_folder,
                                     (foldername+'.txt'))
    station_dict[foldername] = {}
    station_dict[foldername]['folderpath'] = iter_path_to_folder
    station_dict[foldername]['txtpath'] = iter_path_to_txts
    
    with open(iter_path_to_txts) as f:
        txtcontents = f.readlines()
        # isolating just the stream URL link
        splitafterstream = txtcontents[1].split('stream=')
        splitonnewline = splitafterstream[1].split('\n')
        isoed_streamlink = splitonnewline[0]
        station_dict[foldername]['streamlink'] = isoed_streamlink

##### Demonstration of how info may be retrieved from 'station_dict'  for a given station:

In [139]:
station_dict['MN_KXXR']['streamlink']

'https://radio.garden/api/ara/content/listen/LOYtZb13/channel.mp3?1624605094130'

In [140]:
station_dict['MN_KCMP']['folderpath']

'/Users/michaelfelzan/Desktop/RadioTestRun/FiveStations/MN_KCMP'

In [141]:
station_dict['MN_KCMP']['txtpath']

'/Users/michaelfelzan/Desktop/RadioTestRun/FiveStations/MN_KCMP/MN_KCMP.txt'

# { { { { { { { {   {  { { { { { {  { { {  { { { { { {_  Section B.   _} } } } } } } } } }  } } }  } } } } } } } } } } } }   }   }    } 

### Creating designated folders within each station folder to contain mp3 recordings and 'cue files':

In [142]:
for station in sortedstations:
    iterfoldpath = station_dict[station]['folderpath']
    mp3foldpath = os.path.join(iterfoldpath, "mp3_recordings")
    # 'cue' files are necessary for VLC module to run....i think
    cuefoldpath = os.path.join(iterfoldpath, "cue_files")
    
    # These are commented out b/c I already made the directories;
    # Uncomment the following two lines if youd like to create these
    # directories.
    
    #os.mkdir(mp3foldpath)
    #os.mkdir(cuefoldpath)

## Setting up function which uses the VLC module to record 12 second mp3 clips from a radio station, saves mp3s to disc:

##### Functions which initialize VLC instance/player:

In [143]:
def InitializeVLCCue(name,instream,audiofile):
    """Initializes the VLC cue file.
    
    Parameters
    ------------
    name : str
        Name of cue file. Should be formatted like:
        <stationname>_<formattedtime>_CUE  (no extension)
    instream : str
        URL link to radio station stream.
    audiofile : str
        Name of mp3 file. Should be formatted like:
        <stationname>_<formattedtime>_.mp3
        """
    cueout = '%s.cue'%(name)
    with open(cueout, 'w') as outf_writer:
        outf_writer.write('PERFORMER "%s"\n'%(instream))
        outf_writer.write('TITLE "%s"\n' %(name))
        outf_writer.write('FILE "%s" WAVE\n' %(audiofile))

        
def InitializeVLCPlayer(instream, audiofile):
    """Initializes a VLC player which plays 
    locally and saves to an mp3file.
    
    Parameters
    ------------
    instream : str
        URL link to radio station stream.
    audiofile : str
        Name of mp3 file. Should be formatted like:
        <stationname>_<formattedtime>_.mp3
        """
    inst = vlc.Instance()   
    p = inst.media_player_new()
    
    # default stream output chain
    cmd1 = "sout=#duplicate{dst=file{dst=%s},dst=display}" %(audiofile)
    # stream output
    cmd2 ="no-sout-rtp-sap"
    # filename for stream
    cmd3 = "no-sout-standard-sap"
    # play locally while streaming it
    cmd4 ="sout-keep"
    # https://wiki.videolan.org/VLC_command-line_help
    
    med=inst.media_new(instream,cmd1,cmd2,cmd3,cmd4) 
    # get media resource locator
    med.get_mrl()
    med.add_option('start-time=0.0')
    p.set_media(med)
    
    return p, med

##### Building function which utilizes above functions, to record 12 second mp3 clip from an inputted radio stream:

In [153]:
def RadioMP3Recorder(cuename, instream, audiofile):
    """Function that utilizes the InitializeVLCCue() and
    InitializeVLCPlayer() functions, in order to sample
    a 12-second mp3 clip from an inputted radio stream.
    Parameters are the same as above two functions:
    
    Parameters
    ------------
    cuename : str
        Name of cue file. Should be formatted like:
        <stationname>_<formattedtime>_CUE  (no extension)
    instream : str
        URL link to radio station stream.
    audiofile : str
        Name of mp3 file. Should be formatted like:
        <stationname>_<formattedtime>_.mp3
        """
    InitializeVLCCue(cuename, instream, audiofile)
    p,med = InitializeVLCPlayer(instream, audiofile)
    p.play()
    # 30 second sleep period between playing stream and
    # stopping is needed needed because function must wait
    # ~15 seconds before recording, in order to bypass 
    # possible ads, and return a long enough (~12sec) clip.
    time.sleep(40)
    p.stop()

## ACRCloud API Route Configuration

##### Configuring ACR Cloud song recognizer function:

In [145]:
acrcloud_config = {
    'host':'identify-eu-west-1.acrcloud.com',
    'access_key': acrcloud_accesskey,
    'access_secret': acrcloud_secretkey,
    'recognize_type': ACRCloudRecognizeType.ACR_OPT_REC_AUDIO, # could be 'humming audio' as well
    'debug':False,
    'timeout':5 # seconds
}

# ACRCloudRecognizer was an imported at the start of notebook
ACR_recognizer = ACRCloudRecognizer(acrcloud_config)


def RouteToACRCloud(mp3):
    """Function that routes an mp3 file (path) to the
    ACRCloud API, in order to return the song name info.
    
    Parameters
    ------------
    mp3 : str
        Name of mp3 file (path)
        """
    buf = open(mp3,'rb').read()
    # start second will be 0; will record sampled mp3 for 10 seconds
    songread_output = ACR_recognizer.recognize_by_filebuffer(buf, 30)
    return songread_output

In [156]:
for station in station_dict:
    timerightnow = datetime.datetime.now()
    #print(timerightnow)
    formattedtime = timerightnow.strftime("%Y_%m_%d_T%H_%M_%S")
    #print(formattedtime)
    
    input_stream = station_dict[station]['streamlink']
    print(input_stream)
    print(type(input_stream))
    input_cuename = station+'_'+formattedtime+'_CUE'
    print(input_cuename)
    print(type(input_cuename))
    input_mp3name = station+'_'+formattedtime+'.mp3'
    print(input_mp3name)
    print(type(input_mp3name))

https://radio.garden/api/ara/content/listen/3WobtoSX/channel.mp3?1624605251991
<class 'str'>
MN_KCMP_2021_06_25_T02_35_13_CUE
<class 'str'>
MN_KCMP_2021_06_25_T02_35_13.mp3
<class 'str'>
https://radio.garden/api/ara/content/listen/bTYyl91D/channel.mp3?1624605308486
<class 'str'>
MN_KQRS_2021_06_25_T02_35_13_CUE
<class 'str'>
MN_KQRS_2021_06_25_T02_35_13.mp3
<class 'str'>
http://18253.live.streamtheworld.com/KSTPFMAAC.aac?dist=hubbard&source=hubbard-web&ttag=web&gdpr=0
<class 'str'>
MN_KSTP_2021_06_25_T02_35_13_CUE
<class 'str'>
MN_KSTP_2021_06_25_T02_35_13.mp3
<class 'str'>
http://radio.garden/api/ara/content/listen/gs47Xnpy/channel.mp3?1618447430795
<class 'str'>
MN_KUOM_2021_06_25_T02_35_13_CUE
<class 'str'>
MN_KUOM_2021_06_25_T02_35_13.mp3
<class 'str'>
https://radio.garden/api/ara/content/listen/LOYtZb13/channel.mp3?1624605094130
<class 'str'>
MN_KXXR_2021_06_25_T02_35_13_CUE
<class 'str'>
MN_KXXR_2021_06_25_T02_35_13.mp3
<class 'str'>


# { { { { { { { {   {  { { { { { {  { { {  { { { { { {_  Section C.   _} } } } } } } } } }  } } }  } } } } } } } } } } } }   }   }    } 

## Creating function which cycles through every radio station, records 12 second .mp3 audio clip, routes audio clip to ACR Cloud's song identifier API, and then routes the song name to the Discogs API, to retrieve the genre information for that song. 

### Each mp3 recording is named based on the station name and the time the mp3 was recorded. Once the .mp3 has been recorded/outputted, it is shuttled into each station's respective folder's "mp3_recordings" folder. 

### If ACR Cloud was able to ID the song, and the Discogs API was able to retrieve song genre information for that song name, the mp3 filename (which includes the time it was recorded), the song name, artist, album, and genre(s) are appended to that stations .txt file. 

In [146]:
def RadioStationCycler():
    """Function that utilizes the previously created
    RadioMP3Recorder() function, to sample ~12 second
    mp3 audio clips from every radio station in the
    radio stations list. The mp3 clip on a given iteration
    (for a given station) is routed to ACRCloud to return
    the track name, artist, and album. If nothing is returned,
    the function prints a message ("No ACRCloud return"), and
    then skips to the next station. If there is a valid ACRcloud
    return, the track info is then routed to the Discogs API
    to return genre/style information. If there is a valid
    return from the Discogs API, the genre info is printed
    on the console, but more importantly, it is written in
    that given staton's .txt file. Because the mp3 recording
    names include the date/time they were recorded, this
    info is stored alongside the track info in the .txt.
    
    NOTE: Right now, the function is set to iterate through
    all of the stations in the stations list, and does not
    stop until every station stream has been recorded. If
    the 'Interrupt Kernel' button is pressed, the last radio
    stream the function left off on will continue to play
    until the terminal running the notebook is shut down (lol)
    """
    for station in station_dict:
        timerightnow = datetime.datetime.now()
        formattedtime = timerightnow.strftime("%Y_%m_%d_T%H_%M_%S")
    
        input_stream = station_dict[station]['streamlink']
        input_cuename = station+'_'+formattedtime+'_CUE'
        input_mp3name = station+'_'+formattedtime+'.mp3'
    
        print(f'~~Now playing... {station}~~')
        RadioMP3Recorder(input_cuename,
                         input_stream,
                         input_mp3name)
    
        pathtocue = os.path.join(workindir,(input_cuename+'.cue'))
        pathtomp3 = os.path.join(workindir,input_mp3name)
        pathtocuedest = os.path.join(workindir,'FiveStations',station,'cue_files')
        pathtomp3dest = os.path.join(workindir,'FiveStations',station,'mp3_recordings')
    
        shutil.move(pathtocue, pathtocuedest)
        shutil.move(pathtomp3, pathtomp3dest)
    
        itersongpath = pathtomp3dest + '/' + input_mp3name
        acrcloud_output = RouteToACRCloud(itersongpath)
        songread_dict = json.loads(acrcloud_output)
    
        if songread_dict['status']['msg'] == 'No result':
            print(f'No ACRCloud return from {input_mp3name}, skipping to next station')
            pass
        elif songread_dict['status']['msg'] == 'May Be Mute':
            print(f'No ACRCloud return from {input_mp3name}, skipping to next station')
            pass
        elif songread_dict['status']['msg'] == 'Decode Audio Error':
            print(f'No ACRCloud return from {input_mp3name}, skipping to next station')
            pass
        else:
            songtitle = songread_dict['metadata']['music'][0]['title']
            songartist = songread_dict['metadata']['music'][0]['artists'][0]['name']
            songalbum = songread_dict['metadata']['music'][0]['album']['name']
            print(f'{input_mp3name} is the song {songtitle} by {songartist} from the album {songalbum}')
        
            percent20song = songtitle+'%20'+songartist+'%20'+songalbum
            discogs_input = percent20song.replace(" ", "%20")
            discogs_req = ('https://api.discogs.com/database/search?q=' + 
                           discogs_input + 
                           '&key=' + 
                           client_key + 
                           '&secret=' +
                           client_secret)
            discogs_req_obj = requests.get(discogs_req)
            discogs_songinfo_dict = json.loads(discogs_req_obj.content.decode('utf-8'))
        
            if discogs_songinfo_dict['results'] == []:
                print(f'No Discogs return from {input_mp3name}, skipping to next station')
                pass
            else:
                style_list = discogs_songinfo_dict['results'][0]['style']
                genre_list = discogs_songinfo_dict['results'][0]['genre']
                styleliststring = ','.join(style_list)
                genreliststring = ','.join(genre_list)
            
                with open(station_dict[station]['txtpath'], "a+") as file_object:
                    file_object.seek(0)
                    data = file_object.read(100)
                    if len(data) > 0 :
                        file_object.write("\n")
                    file_object.write((input_mp3name+
                                       ','+
                                       'song='+songtitle+','+
                                       'artist='+songartist+','+
                                       'album='+songalbum+','+
                                       #'genres='+genreliststring+','+
                                       'styles='+styleliststring))
                print(f'Discogs returned the genre(s): {styleliststring} for this track.')

In [151]:
for station in station_dict:
    timerightnow = datetime.datetime.now()
    formattedtime = timerightnow.strftime("%Y_%m_%d_T%H_%M_%S")
    
    input_stream = station_dict[station]['streamlink']
    input_cuename = station+'_'+formattedtime+'_CUE'
    input_mp3name = station+'_'+formattedtime+'.mp3'
    
    print(f'~~Now playing... {station}~~')
    RadioMP3Recorder(input_cuename,
                     input_stream,
                     input_mp3name)
    
    pathtocue = os.path.join(workindir,(input_cuename+'.cue'))
    pathtomp3 = os.path.join(workindir,input_mp3name)
    pathtocuedest = os.path.join(workindir,'FiveStations',station,'cue_files')
    
    print(pathtocue)
    print(pathtomp3)
    print(pathtocuedest)

~~Now playing... MN_KCMP~~
/Users/michaelfelzan/Desktop/RadioTestRun/MN_KCMP_2021_06_25_T02_25_00_CUE.cue
/Users/michaelfelzan/Desktop/RadioTestRun/MN_KCMP_2021_06_25_T02_25_00.mp3
/Users/michaelfelzan/Desktop/RadioTestRun/FiveStations/MN_KCMP/cue_files
~~Now playing... MN_KQRS~~
/Users/michaelfelzan/Desktop/RadioTestRun/MN_KQRS_2021_06_25_T02_25_40_CUE.cue
/Users/michaelfelzan/Desktop/RadioTestRun/MN_KQRS_2021_06_25_T02_25_40.mp3
/Users/michaelfelzan/Desktop/RadioTestRun/FiveStations/MN_KQRS/cue_files
~~Now playing... MN_KSTP~~
/Users/michaelfelzan/Desktop/RadioTestRun/MN_KSTP_2021_06_25_T02_26_20_CUE.cue
/Users/michaelfelzan/Desktop/RadioTestRun/MN_KSTP_2021_06_25_T02_26_20.mp3
/Users/michaelfelzan/Desktop/RadioTestRun/FiveStations/MN_KSTP/cue_files
~~Now playing... MN_KUOM~~
/Users/michaelfelzan/Desktop/RadioTestRun/MN_KUOM_2021_06_25_T02_27_00_CUE.cue
/Users/michaelfelzan/Desktop/RadioTestRun/MN_KUOM_2021_06_25_T02_27_00.mp3
/Users/michaelfelzan/Desktop/RadioTestRun/FiveStations/MN

### !! Working directory must be at the base of users working folder in order for RadioStationCycler() function to operate properly !!

In [147]:
os.chdir(workindir)
os.getcwd()

'/Users/michaelfelzan/Desktop/RadioTestRun'

## Running function // printing results

In [148]:
RadioStationCycler()

~~Now playing... MN_KCMP~~
No ACRCloud return from MN_KCMP_2021_06_25_T02_19_20.mp3, skipping to next station
~~Now playing... MN_KQRS~~
No ACRCloud return from MN_KQRS_2021_06_25_T02_20_00.mp3, skipping to next station
~~Now playing... MN_KSTP~~
No ACRCloud return from MN_KSTP_2021_06_25_T02_20_40.mp3, skipping to next station
~~Now playing... MN_KUOM~~
MN_KUOM_2021_06_25_T02_21_20.mp3 is the song The Chalice by Nicole Mitchell from the album Mandorla Awakening II: Emerging Worlds
Discogs returned the genre(s): Post Bop,Avant-garde Jazz,Free Improvisation for this track.
~~Now playing... MN_KXXR~~
No ACRCloud return from MN_KXXR_2021_06_25_T02_22_01.mp3, skipping to next station


# { { { { { { { {   {  { { { { { {  { { {  { { { { { {_  Section D.   _} } } } } } } } } }  } } }  } } } } } } } } } } } }   }   }    } 

## Setting up function which requests a given station's ERP, HAAT, and lat/long using 'get' requests on radio-locator.com :

In [18]:
def RadioTowerInfoGetter(callsign):
    """Function that requests the HTML of
    the radio-locator.com webpage for whatever 
    radio station callsign is inputted into the function.
    
    The function parses the HTML to return a dictionary
    containing the radio stations ERP, lat/long,
    HAAT, HAGL, and HASL.
    
    Parameters
    ------------
    callsign : str
        Name of station callsign (eg. KEXP)
    """
    radiolocheaders = {
        'sec-ch-ua' : '"Google Chrome";v="89", "Chromium";v="89", ";Not A Brand";v="99"',
        'sec-ch-ua-mobile' : '?0',
        'Upgrade-Insecure-Requests' : '1',
        'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.114 Safari/537.36'
    }
    locatorURL = f'https://radio-locator.com/cgi-bin/finder?call={callsign}&x=0&y=0&sr=Y&s=C'
    
    radiolocreq = requests.get(locatorURL,
                               headers=radiolocheaders)
    stationHTML = radiolocreq.content.decode('utf-8')
    stationsoup = BeautifulSoup(stationHTML)
    techvalues = stationsoup.find_all("td",
                                      class_='tech_value')
    
    if techvalues == []:
        print(f"No radio tower info could be retrieved for station {callsign}")
        return False
    else:
        ERP = []
        coords = []
        heights = []
        # Parsing the HTML:
        for item in techvalues:
            for characters in item:
                for sub in characters:
                    if '" N' in sub:
                        coords.append(sub)
            if 'Watts' in characters:
                ERP.append(characters)
            elif 'meters' in characters:
                heights.append(characters)
            else:
                pass
    
        itercoords = coords[0]
        iterERP = ERP[0]
        iterHAAT = heights[0]
        iterHAGL = heights[1]
        iterHASL = heights[2]
    
        towerinfo = {
            'ERP' : iterERP,
            'Coords' : itercoords,
            'HAAT' : iterHAAT,
            'Height Above Ground Level' : iterHAGL,
            'Height Above Sea Level' : iterHASL
        }
    
        return towerinfo

### Example returns from RadioTowerInfoGetter() on a station callsign:

In [19]:
kexptower = RadioTowerInfoGetter('KEXP')
kexptower

{'ERP': '4700 Watts',
 'Coords': '47° 36\' 57" N, 122° 18\' 32" W',
 'HAAT': '211 meters (692 feet)',
 'Height Above Ground Level': '122 meters (400 feet)',
 'Height Above Sea Level': '247 meters (810 feet)'}

In [20]:
kalxtower = RadioTowerInfoGetter('KALX')
kalxtower

{'ERP': '500 Watts',
 'Coords': '37° 52\' 40" N, 122° 14\' 48" W',
 'HAAT': '238 meters (781 feet)',
 'Height Above Ground Level': '18 meters (59 feet)',
 'Height Above Sea Level': '350 meters (1148 feet)'}

## Setting up function which writes the dictionary return from RadioTowerInfoGetter() function to a .txt file in a given station's folder :

In [8]:
def StationInfoTxtWriter(stationfoldername):
    """Function that writes the dictionary return
    from RadioTowerInfoGetter() for a station 
    to a .txt file in that station's corresponding
    folder (only if the RadioTowerInfoGetter()
    successfully returns tower info).
    
    Parameters
    ------------
    stationfoldername : str
        Name of a radio stations folder (eg. CA_KALX)
    """
    call_sign = stationfoldername.split("_")[1]
    towerinfodict = RadioTowerInfoGetter(call_sign)
    if towerinfodict == False:
        print("yes, it failed")
        pass
    else:
        infofilename = os.path.join(workindir,
                                    "STATIONS",
                                    stationfoldername,
                                    f"{call_sign}_towerinfo.txt")
        textyfile = open(infofilename,
                    "w+")
        textyfile.write("{\n")
        for k in towerinfodict.keys():
            textyfile.write("'{}':'{}'\n".format(k, towerinfodict[k]))
        textyfile.write("}")
        textyfile.close()
        print(f"Sucessfully wrote {call_sign}_towerinfo.txt")

### Running the StationInfoTxtWriter() function on all stations:

In [29]:
for station in sortedstations:
    StationInfoTxtWriter(station)
    time.sleep(10)

Sucessfully wrote KALX_towerinfo.txt
No radio tower info could be retrieved for station KLDBLP
yes, it failed
Sucessfully wrote KXSC_towerinfo.txt
Sucessfully wrote WREK_towerinfo.txt
Sucessfully wrote WUOG_towerinfo.txt
Sucessfully wrote WEIU_towerinfo.txt
No radio tower info could be retrieved for station WCRD
yes, it failed
Sucessfully wrote WGRE_towerinfo.txt
Sucessfully wrote WERS_towerinfo.txt
Sucessfully wrote WMBR_towerinfo.txt
Sucessfully wrote WZBC_towerinfo.txt
Sucessfully wrote WMUC_towerinfo.txt
No radio tower info could be retrieved for station WCBN
yes, it failed
No radio tower info could be retrieved for station KUOM
yes, it failed
Sucessfully wrote WKNC_towerinfo.txt
Sucessfully wrote WSOU_towerinfo.txt
Sucessfully wrote WICB_towerinfo.txt
Sucessfully wrote WRHU_towerinfo.txt
Sucessfully wrote WSBU_towerinfo.txt
Sucessfully wrote WVKR_towerinfo.txt
Sucessfully wrote WCSB_towerinfo.txt
Sucessfully wrote KPSU_towerinfo.txt
Sucessfully wrote KWVA_towerinfo.txt
Sucessfully

# { { { { { { { {   {  { { { { { {  { { {  { { { { { {_  Section E.   _} } } } } } } } } }  } } }  } } } } } } } } } } } }   }   }    } 

## The following cell opens a javascript file, using the os.popen() function. The file is called 'FezCC.js' (my nickname is Fez; FCC pun) and it was ripped from the FCC's 'FM and TV curve propagation calculator' webpage ( https://www.fcc.gov/sites/default/files/fm-and-tv-propagation-curves.html ). The script was saved into a new file and re-parameterized so that the user can enter in first the ERP (kW), the HAAT (meters), **then '250' and '-' are just givens,** the signal strength (dBu) at which you'd like to predict the distance you'd be from that station, and then "0" is just another given.

## The output of this script is the distance you'd be from a given radio station in order to be at 60dBu ('local' resolution)

In [22]:
stream = os.popen('node fezCC.js 4.7 211 "250" "-" 60 "0"')
output = (stream.read()).split('\n')[0]
output

'37.699469101964276'

#### Setting up a function that converts degrees minutes seconds lat/long into decimal degrees:

In [9]:
def dms_to_dd(d, m, s):
    """Function that converts degrees minutes seconds
    into decimal degrees.
    
    Parameters
    ------------
    d : str (tho doesnt matter--converted to float in function)
        degree
    m : str (tho doesnt matter--converted to float in function)
        minutes
    s : str (tho doesnt matter--converted to float in function)
        seconds
    """
    dd = d + float(m)/60 + float(s)/3600
    return round(dd, 6)

#### Setting up a function that isolates the degree, minute, and second components of the DMS return from the StationInfoTxtWriter(), so that they may be separately inputted into the dms_to_dd() function parameters:

In [10]:
def DMS_Isolator(dms_string):
    """function that isolates the degree, minute, and second 
    components  of the DMS return from the StationInfoTxtWriter(),
    so that they may be separately inputted into the
    dms_to_dd() function parameters
    """
    degree = dms_string.split("°")[0]
    minutefirstsplit = dms_string.split("'")[0]
    minute = minutefirstsplit.split("° ")[1]
    secondfirstsplit = dms_string.split('"')[0]
    second = secondfirstsplit.split("' ")[1]
    
    return [degree,minute,second]

## Iterating over every station in the radio stations list, extracting the ERP, lat/long, and HAAT from that stations TowerInfo.txt file (if one exists), converts the lat/long into decimal degrees, and also calculates the broadcast distance (at 60dBu) for each station, given their HAAT and ERP. 

## All of this information is then neatly written to a CSV file in that station's folder.

In [176]:
for station in sortedstations:
    pathtostationfold = os.path.join(workindir,
                                    "STATIONS",
                                    station)
    itercallsign = station.split("_")[1]
    stateabv = station.split("_")[0]
    pathtotowertxt = os.path.join(pathtostationfold,
                            f'{itercallsign}_towerinfo.txt')
    pathtotxt = os.path.join(pathtostationfold,
                             f'{station}.txt')
    newinfocsv = os.path.join(pathtostationfold,
                             f'{station}_INFO.csv')
    stationfolditems = os.listdir(pathtostationfold)
    fullpathitems = []
    for item in stationfolditems:
        fullpathitems.append(os.path.join(pathtostationfold,
                                         item))
    if pathtotowertxt in fullpathitems:
        with open(pathtotowertxt) as foo:
            txtz = foo.readlines()
            for item in txtz:
                
                if 'ERP' in item:
                    wattsfirstsplit = item.split(":'")[1]
                    watts = wattsfirstsplit.split(" Watts")[0]
                    kilowatts = (float(watts.replace(',','')))/1000
                    
                elif 'Coords' in item:
                    coordsfirstsplit = item.split(":'")[1]
                    lat_dms = coordsfirstsplit.split(",")[0]
                    long_dms = (coordsfirstsplit.split(", ")[1])[:-2]
                    
                    lat_dms_list = DMS_Isolator(lat_dms)
                    long_dms_list = DMS_Isolator(long_dms)
                    
                    lat_dd = dms_to_dd(float(lat_dms_list[0]),
                                       float(lat_dms_list[1]),
                                       float(lat_dms_list[2]))
                    long_dd = dms_to_dd(float(long_dms_list[0]),
                                        float(long_dms_list[1]),
                                        float(long_dms_list[2]))
                    negative_longdd = long_dd*-1
                    
                elif 'HAAT' in item:
                    HAATfirstsplit = item.split(":'")[1]
                    HAAT_clean = HAATfirstsplit.split(" meters")[0]
                    HAAT_float = float(HAAT_clean)
        
                    stream_param = f'node fezCC.js {kilowatts} {HAAT_float} "250" "-" 60 "0"'
                    stream = os.popen(stream_param)
                    BUFF60DIST = (stream.read()).split('\n')[0]
                
        with open(newinfocsv,'w',newline='') as csvfile:
            fieldnames = ['StateABV',
                          'CALLSIGN',
                          'LAT',
                          'LONG',
                          'ERP',
                          'HAAT',
                          '60dBu_DIST']
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()
            writer.writerow({
                'StateABV': stateabv,
                'CALLSIGN': itercallsign,
                'LAT' : lat_dd,
                'LONG' : negative_longdd,
                'ERP' : kilowatts,
                'HAAT' : HAAT_clean,
                '60dBu_DIST' : BUFF60DIST
            })
                
    else:
        print(f'tower info .txt does not exist for {station}')
        pass

print("\n     *~*~~~~~**~~~~~~~~~~~~~~**~*~*\n"+
      "Successfully wrote all other station tower info CSVs."+
      "\nCSVs outputted to respective station folders."
      "\n   *~*~~~~~**~~~~~~~~~~~~~~**~*~*")

tower info .txt does not exist for CA_KLDBLP
tower info .txt does not exist for IN_WCRD
tower info .txt does not exist for MI_WCBN
tower info .txt does not exist for MN_KUOM

     *~*~~~~~**~~~~~~~~~~~~~~**~*~*
Successfully wrote all other station tower info CSVs.
CSVs outputted to respective station folders.
   *~*~~~~~**~~~~~~~~~~~~~~**~*~*


# { { { { { { { {   {  { { {  {  { { {  { { { { { {_  Section F. (pt1)   _} } } } } } } } } }  } } }  } } } } } } } } } } } }   }       } 

##### (Creating new folder, "StationInfoTables" :

In [47]:
stationinfopath = os.path.join(workindir, "StationInfoTables")
os.mkdir(stationinfopath)

#### COPYING each stations TowerInfo CSV to the StationInfoTables folder:

In [48]:
# COPYING FILES TO StationInfoTables FOLDER

for station in sortedstations:
    pathtostationfold = os.path.join(workindir,
                                    "STATIONS",
                                    station)
    newinfocsv = os.path.join(pathtostationfold,
                             f'{station}_INFO.csv')
    stationfolditems = os.listdir(pathtostationfold)
    fullpathitems = []
    for item in stationfolditems:
        fullpathitems.append(os.path.join(pathtostationfold,
                                         item))
    if newinfocsv in fullpathitems:
        copyfile(newinfocsv,
                 os.path.join(workindir,
                              "StationInfoTables",
                              f'{station}_INFO.csv'))

## Iterating over all of the station folders, and merging each station's TowerInfo CSV contents into one 'master' TowerInfo CSV. This CSV includes every radio station's ERP, HAAT, lat/long, 60 dBu distance, callsign and state abreviation :

In [174]:
for station in sortedstations:
    pathtofold = os.path.join(workindir,
                              "StationInfoTables")
INFOconcatCSV = os.path.join(pathtofold,
                            "STATIONINFOconcat.csv") 
allFiles = glob.glob(pathtofold + "/*.csv")
allFiles.sort()
with open(INFOconcatCSV, 'wb') as outfile:
        for i, fname in enumerate(allFiles):
            with open(fname, 'rb') as infile:
                if i != 0:
                    infile.readline()  # Throw away header on all but first file
            # Block copy rest of file from input to output without parsing
                shutil.copyfileobj(infile, outfile)

print("     *~*~~~~~**~~~~~~~~~~~~~~**~*~*\n"
      "All station tower info CSVs (that exist) have"+
      "\n      been successfully merged together"
      "\n  *~*~~~~~**~~~~~~~~~~~~~~**~*~*")

     *~*~~~~~**~~~~~~~~~~~~~~**~*~*
All station tower info CSVs (that exist) have
      been successfully merged together
  *~*~~~~~**~~~~~~~~~~~~~~**~*~*


## Demonstrating what the 'master' Station Info CSV looks like:

In [177]:
stninfoconcat = os.path.join(workindir,
                             "StationInfoTables",
                             "STATIONINFOconcat.csv")


stninfo_df = pd.read_csv(stninfoconcat)
stninfo_df

,StateABV,CALLSIGN,LAT,LONG,ERP,HAAT,60dBu_DIST
0,CA,KALX,37.877778,-122.246667,0.500,238.00,23.813202
1,CA,KXSC,37.322778,-121.755278,6.000,-47.00,15.752658
2,GA,WREK,33.778056,-84.406111,100.000,102.00,51.246887
3,GA,WUOG,33.949722,-83.382778,26.000,55.00,30.117545
4,IL,WEIU,39.478611,-88.172500,4.000,51.00,18.795361
5,IN,WGRE,39.638611,-86.863611,0.800,54.00,12.783182
6,MA,WERS,42.352222,-71.056389,4.000,186.00,34.379047
7,MA,WMBR,42.361667,-71.083611,0.720,90.00,15.980103
8,MA,WZBC,42.334722,-71.174722,1.000,67.00,14.891976
9,MD,WMUC,38.983056,-76.943333,0.010,1.00,3.151873


### NOTE: Section F Pt. 2 is in a different notebook, because Arcpy was needed (and I didn't want to configure all of the packages on this notebook with Esri's python environment.

### This notebook is titled _____ in the github repo.

### F pt 2 runs through the process of importing this table into ArcGIS, and creating a feature class out of the table using XYTTabletoPoint. The point feature class that is created from the above CSV contains all of the fields of that CSV as attributes. Because of this, a different buffer is able to be created for each individual point representing the broadcast reach. Also, because the station callsigns are included in the attribute table, the radio song play logs are able to be joined on this field.

# { { { { { { { {   {  { { { { { {  { { {  { { { { { {_  Section G.   _} } } } } } } } } }  } } }  } } } } } } } } } } } }   }   }    } 

##### Re-demonstrating how to access a given station's own '.txt' file using the station_dict :

In [52]:
station_dict['WI_WSUM']['txtpath']

'/Users/michaelfelzan/Desktop/5572 RADIO GARDEN/RadioTestRun/STATIONS/WI_WSUM/WI_WSUM.txt'

##### Demonstrating how to open one of these files, and access only the lines of the .txt file that are not a part of the 'header' :

In [127]:
with open(station_dict['WI_WSUM']['txtpath'], "r") as file_object:
    Lines = file_object.readlines()
    cleanLines = []
    txtheader = [0,1,2]
    for i in range(len(Lines)):
        if i not in txtheader:
            cleanLines.append(Lines[i])
    
for line in cleanLines:
    print(line)
    

WI_WSUM_2021_04_15_T00_33_04.mp3,song=Dunkirk,artist=Silverbacks,album=Pink Tide,styles=Indie Rock,Post-Punk

WI_WSUM_2021_04_15_T01_11_40.mp3,song=Press Play,artist=Alex Ebert,album=I vs I,styles=

WI_WSUM_2021_04_15_T01_33_41.mp3,song=Keep On Loving Me,artist=Onra,album=Keep On Loving Me (Edit),styles=House,Disco,Beatdown

WI_WSUM_2021_04_15_T01_54_20.mp3,song=In Rural Virginia; Watching Glowing Lights Crawl from the Dark Corners of the Room,artist=Ricky Eat Acid,album=Three Love Songs,styles=Ambient,Experimental,House

WI_WSUM_2021_04_15_T11_07_31.mp3,song=Porcelain,artist=Moby,album=Porcelain,styles=House,Downtempo

WI_WSUM_2021_04_15_T14_46_39.mp3,song=When I Kissed The Teacher,artist=ABBA,album=More ABBA Gold,styles=Europop,Disco


### Breaking down how each element of the .txt file track play logs will be parsed:

In [160]:
# How to parse genre:

genresplit1 = cleanLines[3].split("styles=")[1]
genrecomma = genresplit1.split('\n')[0]
genrelist = genrecomma.split(',')
genrelist

['Ambient', 'Experimental', 'House']

In [153]:
# How to parse album:

stuffbeforegenre = cleanLines[3].split(",styles")[0]
albumparse = stuffbeforegenre.split('album=')[1]
albumparse

'Three Love Songs'

In [154]:
# How to parse artist:

stuffbeforealbum = stuffbeforegenre.split(",album=")[0]
artistparse = stuffbeforealbum.split(",artist=")[1]
artistparse

'Ricky Eat Acid'

In [155]:
# How to parse song

stuffbeforeartist = stuffbeforealbum.split(",artist=")[0]
songparse = stuffbeforeartist.split(",song=")[1]
songparse

'In Rural Virginia; Watching Glowing Lights Crawl from the Dark Corners of the Room'

In [156]:
# How to parse mp3 filename components:

mp3filename = stuffbeforeartist.split(",song=")[0]
mp3filename

'WI_WSUM_2021_04_15_T01_54_20.mp3'

In [157]:
mp3_components = mp3filename.split('_')
mp3_components[-1] = mp3_components[-1].split('.mp3')[0]
mp3_components

['WI', 'WSUM', '2021', '04', '15', 'T01', '54', '20']

In [158]:
# asigning vars to each mp3_component list item:

stateabrev = mp3_components[0]
radiocallsgn = mp3_components[1]
captureyear = mp3_components[2]
capturemonth = mp3_components[3]
captureday = mp3_components[4]
capturehour = mp3_components[5].split('T')[1]
captureminute = mp3_components[6]
capturesecond = mp3_components[7]

In [ ]:
genrelist = genrecomma.split(',')
            genrecommastr = ",".join(genrelist

In [162]:
# Demonstrating / printing all of the variables parsed from the above functions:

print('STATE :',stateabrev)
print('CALLSIGN :',radiocallsgn)
print('SONG NAME :',songparse)
print('ARTIST NAME :',artistparse)
print('ALBUM NAME :',albumparse)
print('STYLES :',genrecomma)
print('YEAR OF CAPTURE :',captureyear)
print('MONTH OF CAPTURE:',capturemonth)
print('DAY OF CAPTURE :',captureday)
print('HOUR OF CAPTURE :',capturehour)
print('MINUTE OF CAPTURE :',captureminute)
print('SECOND OF CAPTURE :',capturesecond)

STATE : WI
CALLSIGN : WSUM
SONG NAME : In Rural Virginia; Watching Glowing Lights Crawl from the Dark Corners of the Room
ARTIST NAME : Ricky Eat Acid
ALBUM NAME : Three Love Songs
STYLES : Ambient,Experimental,House
YEAR OF CAPTURE : 2021
MONTH OF CAPTURE: 04
DAY OF CAPTURE : 15
HOUR OF CAPTURE : 01
MINUTE OF CAPTURE : 54
SECOND OF CAPTURE : 20


## Iterating over every station .txt and creating a CSV containing the song play log of that station. The fields of the CSV are those in the above cell. The CSV is named based on the day the cell is run, so if the function is run two times in one day, it will overwrite the previous output.

In [178]:
TrackPlayLogDict = {}

for station in sortedstations:
    
    timerightnow = datetime.datetime.now()
    YMD_rightnow = timerightnow.strftime("%Y_%m_%d")
    
    TrackPlayLogDict[station] = {}
    tracklogCSVpath = os.path.join(station_dict[station]['folderpath'],
                                   station +
                                   "_TrackLog_" +
                                   YMD_rightnow +
                                   ".csv")
    
    
    with open(station_dict[station]['txtpath'], "r") as file_object:
        Lines = file_object.readlines()
        cleanLines = []
        txtheader = [0,1,2]
        for i in range(len(Lines)):
            if i not in txtheader:
                cleanLines.append(Lines[i])
        
    with open(tracklogCSVpath,'w',newline='') as csvfile:
        fieldnames = ['stateabrev',
                      'radiocallsgn',
                      'trackname',
                      'trackartist',
                      'trackalbum',
                      'trackstyles',
                      'trackmp3filename',
                      'captureyear',
                      'capturemonth',
                      'captureday',
                      'capturehour',
                      'captureminute',
                      'capturesecond']
        
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        
        for trackplaystring in cleanLines:
            
            genresplit1 = trackplaystring.split("styles=")[1]
            genrecomma = genresplit1.split('\n')[0]
            genrelist = genrecomma.split(',')
            genrecommastr = ",".join(genrelist)
            
            stuffbeforegenre = trackplaystring.split(",styles")[0]
            albumparse = stuffbeforegenre.split('album=')[1]
            
            stuffbeforealbum = stuffbeforegenre.split(",album=")[0]
            artistparse = stuffbeforealbum.split(",artist=")[1]

            stuffbeforeartist = stuffbeforealbum.split(",artist=")[0]
            songparse = stuffbeforeartist.split(",song=")[1]
            
            mp3filename = stuffbeforeartist.split(",song=")[0]
            
            mp3_components = mp3filename.split('_')
            mp3_components[-1] = mp3_components[-1].split('.mp3')[0]
            
            stateabrev = mp3_components[0]
            radiocallsgn = mp3_components[1]
            captureyear = mp3_components[2]
            capturemonth = mp3_components[3]
            captureday = mp3_components[4]
            capturehour = mp3_components[5].split('T')[1]
            captureminute = mp3_components[6]
            capturesecond = mp3_components[7]
            
            writer.writerow({
                'stateabrev' : stateabrev,
                'radiocallsgn' : radiocallsgn,
                'trackname' : songparse,
                'trackartist' : artistparse,
                'trackalbum' : albumparse,
                'trackstyles' : genrecommastr,
                'trackmp3filename' : mp3filename,
                'captureyear' : captureyear,
                'capturemonth' : capturemonth,
                'captureday' : captureday,
                'capturehour' : capturehour,
                'captureminute' : captureminute,
                'capturesecond' : capturesecond
            })
print("*~*~~~~~**~~~~~~~~~~~~~~**~*~*\n"
      "Sucessfully printed track play log CSVs for all stations."+
      "\nAll CSVs outputted to respective station folders."+
      f"\nThis batch of CSVs have the extension: {YMD_rightnow}.csv"+
      "\n*~*~~~~~**~~~~~~~~~~~~~~**~*~*")

*~*~~~~~**~~~~~~~~~~~~~~**~*~*
Sucessfully printed track play log CSVs for all stations.
All CSVs outputted to respective station folders.
This batch of CSVs have the extension: 2021_05_06.csv
*~*~~~~~**~~~~~~~~~~~~~~**~*~*


## Demonstrating what a TrackLog CSV would look like for a given staton :

In [165]:
May6_KALX_log = os.path.join(station_dict['CA_KALX']['folderpath'],
                             "CA_KALX_TrackLog_2021_05_06.csv")
May6_KALX_log

'/Users/michaelfelzan/Desktop/5572 RADIO GARDEN/RadioTestRun/STATIONS/CA_KALX/CA_KALX_TrackLog_2021_05_06.csv'

In [166]:
may6_kalx_df = pd.read_csv(May6_KALX_log)
may6_kalx_df

,stateabrev,radiocallsgn,trackname,trackartist,trackalbum,trackstyles,trackmp3filename,captureyear,captureday,capturehour,captureminute,capturesecond
0,CA,KALX,"Finally (12"" Choice Mix)",CeCe Peniston,Grand House Classics 1,"House,Deep House,Garage House",CA_KALX_2021_04_15_T00_17_08.mp3,2021,15,0,17,8
1,CA,KALX,Le coeur grenadine,Laurent Voulzy,Le coeur grenadine,Chanson,CA_KALX_2021_04_15_T01_34_13.mp3,2021,15,1,34,13
2,CA,KALX,Aviateur,Véronique Jannot,Aviateur,"Synth-pop,Chanson,Italo-Disco",CA_KALX_2021_04_15_T01_38_12.mp3,2021,15,1,38,12
3,CA,KALX,Deaf,Just Mustard,Wednesday,Shoegaze,CA_KALX_2021_04_15_T10_37_23.mp3,2021,15,10,37,23
4,CA,KALX,The Pull,Microphones,"It Was Hot, We Stayed in the Water",Lo-Fi,CA_KALX_2021_04_15_T10_51_33.mp3,2021,15,10,51,33
5,CA,KALX,Here Comes The Sun,Richie Havens,Live At The Cellar Door and at the Santa Monic...,"Folk Rock,Acoustic",CA_KALX_2021_04_15_T14_03_16.mp3,2021,15,14,3,16
6,CA,KALX,Lovin's for Fools,Front Country,Sake of the Sound,"Bluegrass,Folk",CA_KALX_2021_04_15_T21_00_08.mp3,2021,15,21,0,8
7,CA,KALX,Single Saturday Night,Cole Swindell,Single Saturday Night,Country,CA_KALX_2021_04_16_T00_52_26.mp3,2021,16,0,52,26
8,CA,KALX,Bad Case,Lukas Nelson & Promise of the Real,Turn Off The News (Build A Garden),NaN,CA_KALX_2021_04_29_T11_31_05.mp3,2021,29,11,31,5
9,CA,KALX,I Never Cared For You,Willie Nelson,A Horse Called Music,Country,CA_KALX_2021_04_29_T13_19_17.mp3,2021,29,13,19,17
